In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.patches import Circle, RegularPolygon
from matplotlib.path import Path
from matplotlib.projections import register_projection
from matplotlib.projections.polar import PolarAxes
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D


def radar_factory(num_vars, frame='circle'):
    """
    Create a radar chart with `num_vars` Axes.

    This function creates a RadarAxes projection and registers it.

    Parameters
    ----------
    num_vars : int
        Number of variables for radar chart.
    frame : {'circle', 'polygon'}
        Shape of frame surrounding Axes.

    """
    # calculate evenly-spaced axis angles
    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)

    class RadarTransform(PolarAxes.PolarTransform):

        def transform_path_non_affine(self, path):
            # Paths with non-unit interpolation steps correspond to gridlines,
            # in which case we force interpolation (to defeat PolarTransform's
            # autoconversion to circular arcs).
            if path._interpolation_steps > 1:
                path = path.interpolated(num_vars)
            return Path(self.transform(path.vertices), path.codes)

    class RadarAxes(PolarAxes):

        name = 'radar'
        PolarTransform = RadarTransform

        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            
            self.set_theta_zero_location('N')

        def fill(self, *args, closed=True, **kwargs):
            """Override fill so that line is closed by default"""
            return super().fill(closed=closed, *args, **kwargs)

        def plot(self, *args, **kwargs):
            """Override plot so that line is closed by default"""
            lines = super().plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)

        def _close_line(self, line):
            x, y = line.get_data()
            
            if x[0] != x[-1]:
                x = np.append(x, x[0])
                y = np.append(y, y[0])
                line.set_data(x, y)

        def set_varlabels(self, labels):
            self.set_thetagrids(np.degrees(theta), labels)

        def _gen_axes_patch(self):
           
            if frame == 'circle':
                return Circle((0.5, 0.5), 0.5)
            elif frame == 'polygon':
                return RegularPolygon((0.5, 0.5), num_vars,
                                      radius=.5, edgecolor="k")
            else:
                raise ValueError("Unknown value for 'frame': %s" % frame)

        def _gen_axes_spines(self):
            if frame == 'circle':
                return super()._gen_axes_spines()
            elif frame == 'polygon':
                .
                spine = Spine(axes=self,
                              spine_type='circle',
                              path=Path.unit_regular_polygon(num_vars))
              
                spine.set_transform(Affine2D().scale(.5).translate(.5, .5)
                                    + self.transAxes)
                return {'polar': spine}
            else:
                raise ValueError("Unknown value for 'frame': %s" % frame)

    register_projection(RadarAxes)
    return theta


def example_data():
   
    
    data = [
        ['LR', 'RR', 'LR', 'EN', 'RF', 'SVR', 'KNN', 'XGB', 'CB','GB'],
        ('Train-test R2', [
            [0.669 ,0.787 ,0.648 ,0.863 ,0.285 ,0.533 ,-0.321 ,0.23 ,0.517 ,0.604 ],
            [0.123 ,0.121 ,0.068 ,0.080 ,-0.010 ,-0.037 ,-0.081 ,-0.105 ,0.017 ,0.180],
            [0 ,0.449 ,0.229 ,0.244 ,0.157 ,0.035 ,0.417 ,-0.038 ,0.304 ,0.367],]),
        ('Train-test RMSE', [
            [0.146 ,0.117 ,0.151 ,0.094 ,0.215 ,0.174 ,0.292 ,0.223 ,0.177 ,0.16 ],
            [0.238 ,0.238 ,0.246 ,0.244 ,0.256 ,0.259 ,0.264 ,0.267 ,0.252 ,0.230],
            [0 ,0.189 ,0.223 ,0.221 ,0.233 ,0.250 ,0.194 ,0.259 ,0.212 ,0.202  ],]),
        ('Validacija R2', [
            [0 ,0. ,0,0 ,0 ,0 ,0 ,0 ,0 ,0],
            [0 ,0. ,0,0 ,0 ,0 ,0 ,0 ,0 ,0 ],
            [0 ,0. ,0,0 ,0 ,0 ,0 ,0 ,0 ,0],]),
        ('Validacija RMSE', [
            [0.062 ,0.087 ,0.098 ,0.077 ,0.107 ,0.099 ,0.114 ,0.124 ,0.099 ,0.104],
            [0.126 ,0.130 ,0.157 ,0.167 ,0.152 ,0.202 ,0.209 ,0.133 ,0.145 ,0.132],
            [0,0.091 ,0.098 ,0.090 ,0.101 ,0.124 ,0.126 ,0.113 ,0.110 ,0.110 ],]),
        
    ]
    return data


if __name__ == '__main__':
    N = 10
    theta = radar_factory(N, frame='polygon')

    data = example_data()
    spoke_labels = data.pop(0)

    fig, axs = plt.subplots(figsize=(9, 9), nrows=2, ncols=2,
                            subplot_kw=dict(projection='radar'))
    fig.subplots_adjust(wspace=0.46, hspace=0.46, top=0.85, bottom=0.05)

    colors = ['b', 'r', 'g', 'm', 'y']
    # Plot the four cases from the example data on separate Axes
    for ax, (title, case_data) in zip(axs.flat, data):
        ax.set_rgrids([0.2, 0.4, 0.6, 0.8])
        ax.set_title(title, weight='bold', size='medium', position=(0.5, 1.1),
                     horizontalalignment='center', verticalalignment='center')
        
        ax.spines['polar'].set_visible(False)
        for d, color in zip(case_data, colors):
            ax.plot(theta, d, color=color)
            ax.fill(theta, d, facecolor=color, alpha=0.25, label='_nolegend_')
        ax.set_varlabels(spoke_labels)

    # add legend relative to top-left plot
    labels = ('Morgan', 'AtomPair', 'MACCS')
    legend = axs[0, 0].legend(labels, loc=(0.999,1.1),
                              labelspacing=0.1, fontsize='small')

    fig.text(0.5, 0.965, 'Podatkovna množica 1',
             horizontalalignment='center', color='black', weight='bold',
             size='large')

    #plt.savefig(r"C:\Users\matic\OneDrive\Desktop\radar_plot_podatki1.png")
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sample data for 10 datasets and 10 models
datasets_1 = ["PM 1", "PM 2", "PM 3", 
               "PM 4", "PM 5", 
               "PM 6", "PM 7", 
               "PM 8", "PM 9", 
               "PM 10"]

models_1 = ['LR', 'RR', 'LR', 'EN', 'RF', 'SVR', 'KNN', 'XGB', 'CB','GB']

# Actual harvest data for Heatmap 1 (10x10)
harvest_1 = np.array([
    [0 ,0.189 ,0.223 ,0.221 ,0.233 ,0.250 ,0.194 ,0.259 ,0.212 ,0.202],
    [0,1.142 ,1.019 ,1.070 ,0.850 ,1.229 ,1.286 ,0.827 ,0.844 ,0.845 ],
    [0,0.413 ,0.709 ,0.491 ,0.381 ,0.686 ,0.994 ,0.432 ,0.400 ,0.411 ],
    [0 ,0.834 ,0.537 ,0.549 ,0.565 ,0.578 ,0.770 ,0.571 ,0.536 ,0.578 ],
    [0 ,0.985 ,1.016 ,1.041 ,0.886 ,1.229 ,1.008 ,0.853 ,0.709 ,0.793 ],
    [0,0.662 ,0.677 ,0.685 ,0.751 ,0.864 ,0.994 ,0.736 ,0.726 ,0.743],
    [0.494 ,0.424 ,0.374 ,0.382 ,0.462 ,0.545 ,0.638 ,0.411 ,0.383 ,0.408],
    [0 ,0.514 ,0.480 ,0.489 ,0.392 ,0.467 ,0.547 ,0.405 ,0.390 ,0.374 ],
    [0.280 ,0.316 ,0.266 ,0.277 ,0.209 ,0.317 ,0.360 ,0.256 ,0.218 ,0.249],
    [0.673 ,0.659 ,0.535 ,0.579 ,0.575 ,0.995 ,1.101 ,0.656 ,0.643 ,0.579]
])

# Sample data for Heatmap 2
datasets_2 = ["PM 1", "PM 2", "PM 3", 
               "PM 4", "PM 5", 
               "PM 6", "PM 7", 
               "PM 8", "PM 9", 
               "PM 10"]

models_2 = ['LR', 'RR', 'LR', 'EN', 'RF', 'SVR', 'KNN', 'XGB', 'CB','GB']

# Actual harvest data for Heatmap 2 (10x10)
harvest_2 = np.array([
    [0 ,0.091 ,0.098 ,0.090 ,0.101 ,0.124 ,0.126 ,0.113 ,0.110 ,0.110 ],
    [0 ,1.346 ,1.136 ,1.174 ,0.939 ,1.146 ,1.215 ,0.943 ,0.960 ,0.974 ],
    [0 ,0.758 ,0.591 ,0.589 ,0.568 ,0.918 ,1.109 ,0.566 ,0.564 ,0.567],
    [0,1.100 ,0.755 ,0.814 ,0.633 ,0.761 ,0.875 ,0.657 ,0.649 ,0.656],
    [0 ,1.007 ,0.867 ,0.947 ,0.904 ,1.170 ,1.229 ,0.862 ,0.857 ,0.874 ],
    [0 ,0.656 ,0.589 ,0.609 ,0.586 ,0.744 ,0.731 ,0.586 ,0.573 ,0.547],
    [0 ,0.457 ,0.462 ,0.456 ,0.491 ,0.621 ,0.650 ,0.477 ,0.477 ,0.453],
    [0 ,0.502 ,0.467 ,0.473 ,0.399 ,0.464 ,0.558 ,0.391 ,0.393 ,0.389],
    [0.212 ,0.225 ,0.220 ,0.214 ,0.219 ,0.260 ,0.436 ,0.216 ,0.202 ,0.218],
    [0.967 ,0.544 ,0.372 ,0.403 ,0.369 ,0.927 ,0.969 ,0.443 ,0.416 ,0.394 ]
])

# Create a figure with 2 subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))  # Increased figure size

# Heatmap 1 with YlGnBu color map
im1 = ax1.imshow(harvest_1, cmap='YlGnBu', aspect='equal')  # Set aspect to 'equal' for square shapes
ax1.set_xticks(np.arange(len(models_1)), labels=models_1)
ax1.set_yticks(np.arange(len(datasets_1)), labels=datasets_1)
plt.setp(ax1.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

# Loop over data dimensions and create text annotations for heatmap 1
for i in range(len(datasets_1)):
    for j in range(len(models_1)):
        text = ax1.text(j, i, f"{harvest_1[i, j]:.3f}", ha="center", va="center", color="w")
#R²
ax1.set_title("MACCS fingerprint Train-test RMSE")

# Heatmap 2 with OrRd color map
im2 = ax2.imshow(harvest_2, cmap='OrRd', aspect='equal')  # Set aspect to 'equal' for square shapes
ax2.set_xticks(np.arange(len(models_2)), labels=models_2)
ax2.set_yticks(np.arange(len(datasets_2)), labels=datasets_2)
plt.setp(ax2.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

# Loop over data dimensions and create text annotations for heatmap 2
for i in range(len(datasets_2)):
    for j in range(len(models_2)):
        text = ax2.text(j, i, f"{harvest_2[i, j]:.3f}", ha="center", va="center", color="w")
#R²
ax2.set_title("MACCS fingerprint Validacija RMSE")

# Adjust layout
fig.tight_layout()

plt.savefig(r"C:\Users\matic\OneDrive\Desktop\heatmap_MACCSRMSE.png")
plt.show()
